## 問題

https://twitter.com/e869120/status/1378115289649348611

## 解説

https://twitter.com/e869120/status/1378527948563615746

In [1]:
#r "nuget: FSharpPlus"
#load "lib.fsx"

Installed Packages FSharpPlus, 1.3.2

In [2]:
open System
open System.Numerics
open System.Collections.Generic
open FSharpPlus
open Adacola.TypicalProblem90

In [3]:
let [<Literal>] Mod = 1_000_000_007

DPを使ってナイーブに実装した場合は小課題1までは解ける

In [4]:
let calcUsingDP N B Cs =
  let dp = Array.init B (fun b -> if b = 0 then 1 else 0)
  (dp, seq { 1 .. N }) ||> Seq.fold (fun dp j ->
    let result = Array.zeroCreate<int> B
    for i in 0 .. B - 1 do
      let i' = 10 * i
      Cs |> List.iter (fun c ->
        let n = (i' + c) % B
        result[n] <- result[n] + dp[i]
        result[n] <- result[n] % Mod)
    result)
  |> head

In [5]:
calcUsingDP 3 7 [1; 4; 9]

3

In [6]:
calcUsingDP 5 2 [1; 4; 9]

81

In [7]:
calcUsingDP 10000 27 [1; 3; 4; 6; 7; 8; 9]

989112238

行列累乗を使って実装した場合は小課題2までは解ける

In [8]:
let int64Mod = int64 Mod
type Matrix = Matrix of int64[,]
with
  member x.Height = let (Matrix a) = x in Array2D.length1 a
  member x.Width = let (Matrix a) = x in Array2D.length2 a
  static member (*) (((Matrix x) as mx), ((Matrix y) as my)) =
    if mx.Width <> my.Height then invalidArg "mx, my" $"mxの幅({mx.Width})とmyの高さ({my.Height})が一致しません"
    Array2D.init mx.Height my.Width (fun i k -> (0L, Seq.init mx.Width id) ||> Seq.fold (fun s j -> (s + (x[i, j] * y[j, k])) % int64Mod))
    |> Matrix
  static member (%) (Matrix x, y: int64) = x |> Array2D.map (flip (%) y) |> Matrix
  member x.Head = let (Matrix a) = x in a[0, 0]
let arrayToMatrix (a: int64[]) = Array2D.init a.Length 1 (fun i j -> a[i]) |> Matrix

In [9]:
let calcUsingMatrix (N: int64) B Cs =
  let arr = Array2D.zeroCreate B B
  for i in 0 .. B - 1 do
    let i' = 10 * i
    Cs |> List.iter (fun c -> let n = (i' + c) % B in arr[i, n] <- arr[i, n] + 1L)
  let m = Matrix arr
  let initialVector = Array.init B (fun i -> if i = 0 then 1L else 0L) |> arrayToMatrix
  (initialVector, (N, m)) |> Seq.unfold (fun (v, (n, m)) ->
    match n with
    | 0L -> None
    | _ ->
      let v = if n % 2L = 0L then v else m * v
      Some(v.Head |> int, (v, (n / 2L, m * m))))
  |> Seq.last

In [10]:
calcUsingMatrix 3 7 [1; 4; 9]

3

In [11]:
calcUsingMatrix 5 2 [1; 4; 9]

81

In [12]:
calcUsingMatrix 10000 27 [1; 3; 4; 6; 7; 8; 9]

989112238

In [13]:
calcUsingMatrix 1000000000000000000L 29 [1; 2; 4; 5; 7; 9]

853993813

ダブリング的に高速化して小課題3も解ける  
わからなかったので模範コードを参考にして移植

In [14]:
let calcUsingDoubling (N: int64) (B: int) Cs =
  let range = 63
  let int64B = int64 B
  let int64Mod = int64 Mod
  // 前計算
  let power10 = Array.init range (fun i -> modPow 10L (1L <<< i) int64B)
  let getn i (j: int64) (k: int64) = (j * power10[i] + k) % int64B |> int
  // dp[0][i]を求める
  let dp = Array2D.zeroCreate<int64> range B
  Cs |> List.iter (fun c -> let n = c % B in dp[0, n] <- dp[0, n] + 1L)
  // dp[1][i], dp[2][i], ..., dp[2^n][i]を求める
  for i in 0 .. range - 2 do
    for j in 0 .. B - 1 do
      for k in 0 .. B - 1 do
        let n = getn i j k
        dp[i + 1, n] <- (dp[i + 1, n] + dp[i, j] * dp[i, k]) % int64Mod
  // 繰り返し二乗法によりdp[N][i]を求める
  let answer = Array2D.zeroCreate<int64> range B
  answer[0, 0] <- 1L
  for i in 0 .. range - 2 do
    if N &&& (1L <<< i) <> 0L then
      for j in 0 .. B - 1 do
        for k in 0 .. B - 1 do
          let n = getn i j k
          answer[i + 1, n] <- (answer[i + 1, n] + answer[i, j] * dp[i, k]) % int64Mod
    else
      for j in 0 .. B - 1 do answer[i + 1, j] <- answer[i, j]
  answer[range - 1, 0]

In [15]:
calcUsingDoubling 3 7 [1; 4; 9]

3

In [16]:
calcUsingDoubling 5 2 [1; 4; 9]

81

In [17]:
calcUsingDoubling 10000 27 [1; 3; 4; 6; 7; 8; 9]

989112238

In [18]:
calcUsingDoubling 1000000000000000000L 29 [1; 2; 4; 5; 7; 9]

853993813

In [19]:
calcUsingDoubling 1000000000000000000L 957 [1; 2; 3; 5; 6; 7; 9]

205384995